In [1]:
import os
import warnings
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from mne.io import read_raw_edf
import yasa
from preprocessing import crop_hypno, extract_features
import mne
import sleepecg

In [2]:
# Define paths (can be defined in config files)
parent_dir = os.path.dirname(os.getcwd())
eeg_dir = parent_dir+'/data/edfs/shhs2/'
hypno_dir = parent_dir+'/data/annotations-events-profusion/shhs2/'
out_dir = '/output/features/'
if not os.path.isdir(parent_dir+out_dir):
    os.mkdir(parent_dir+out_dir)

In [3]:
df_subj = pd.read_csv(parent_dir+"/output/split/shhs_split.csv")
df_subj = df_subj.query("set == 'training'").set_index("subj")

In [4]:
print(df_subj.shape[0], 'subjects remaining')
df_subj.head(10)

2 subjects remaining


,age,male,bmi,ahi,ethnicity,set,hypertension
subj,,,,,,,
200077,41.0,1,23.388687,9.738220,caucasian,training,0.0
200078,54.0,1,30.211833,19.685039,caucasian,training,1.0


In [13]:
df = []
include = ['EEG', 'EOG(L)', 'EMG']
sf = 100

for sub in tqdm(df_subj.index):
    eeg_file = eeg_dir + 'shhs2-' + str(sub) + '.edf'
    hypno_file = hypno_dir + 'shhs2-' + str(sub) + '-profusion.xml'
    
    # Check that file exists
    if not os.path.isfile(eeg_file):
        warnings.warn("File not found %s" % eeg_file)
        continue
    if not os.path.isfile(hypno_file):
        warnings.warn("File not found %s" % hypno_file)
        continue

    # LOAD EEG DATA
    try:
        raw = read_raw_edf(eeg_file, preload=False, verbose=0)
        raw.drop_channels(np.setdiff1d(raw.info['ch_names'], include))
        # Skip subjects if channel were not found
        assert len(raw.ch_names) == len(include)
        raw.load_data()
    except:
        continue
        
    # Resample and high-pass filter 
    raw.resample(sf, npad="auto")
    
    # LOAD HYPNOGRAM
    hypno, sf_hyp = yasa.load_profusion_hypno(hypno_file)
    # (Optional) We keep up to 15 minutes before / after sleep
    # hypno, tmin, tmax = crop_hypno(hypno)
    # raw.crop(tmin, tmax)
    # Check that hypno and data have the same number of epochs
    n_epochs = hypno.shape[0]
    if n_epochs != np.floor(raw.n_times / sf / 30):
        print("- Hypno and data size do not match.")
        continue
    
    # Convert hypnogram to str
    df_hypno = pd.Series(hypno)
    df_hypno.replace({0: 'W', 1: 'N1', 2: 'N2', 3: 'N3', 4: 'R'}, inplace=True)
    stage_min = df_hypno.value_counts(sort=False) / 2

    # INCLUSION CRITERIA (DISABLED)
    # Hypnogram must include all stages
#     if np.unique(hypno).tolist() != [0, 1, 2, 3, 4]:
#         print("- Not all stages are present.")
#         continue
#     # If the duration is not between 4 to 12 hours, skip subject
#     if not(4 < n_epochs / 120 < 12):
#         print("- Recording too short/long.")
#         continue

    # EXTRACT FEATURES
    features = extract_features(df_subj, sub, raw, include)
    # Add hypnogram
    features['stage'] = df_hypno.to_numpy()
    
    
    # extract ECG features
    # Read ECG data
    # warnings.filterwarnings('ignore')
    raw = mne.io.read_raw(eeg_file)
    raw_data = raw.get_data()
    # you can get the metadata included in the file and a list of all channels:
    info = raw.info
    channels = raw.ch_names

    data, times = raw[:]  

    sf = 256
    heartbeat_times = sleepecg.detect_heartbeats(data[3], sf)/sf
    sleep_stage_duration = 30
    record_duration = heartbeat_times[-1]
    num_stages = record_duration // sleep_stage_duration
    # check if ECG epoch number is the same with EEG epoch number
    if features.shape[0] - num_stages > 1:
        print('Skip due to different numbers of epoch')
        continue
    stage_times = np.arange(num_stages) * sleep_stage_duration
    min_rri, max_rri = None, None
    lookback, lookforward = 240, 60
    rri = sleepecg.preprocess_rri(
            np.diff(heartbeat_times),
            min_rri=min_rri,
            max_rri=max_rri,
        )
    rri_times = heartbeat_times[1:]
    fs_rri_resample = 256
    max_nans = 0.5
    feature_ids = []

    td_feature = sleepecg.feature_extraction._hrv_timedomain_features(rri,
                        rri_times,stage_times,lookback,lookforward,)
    fd_feature = sleepecg.feature_extraction._hrv_frequencydomain_features(rri,rri_times,
                    stage_times,lookback,lookforward,fs_rri_resample, max_nans, feature_ids)
    
    col = ['ECG_meanNN','ECG_maxNN','ECG_minNN','ECG_rangeNN','ECG_SDNN','ECG_RMSSD','ECG_SDSD','ECG_NN50',
       'ECG_NN20','ECG_pNN50','ECG_pNN20','ECG_medianNN','ECG_madNN','ECG_iqrNN','ECG_cvNN',
       'ECG_cvSD','ECG_meanHR','ECG_maxHR', 'ECG_minHR', 'ECG_stdHR',
       'ECG_SD1', 'ECG_SD2', 'ECG_S', 'ECG_SD1_SD2_ratio', 'ECG_CSI', 'ECG_CVI','ECG_total_power', 
       'ECG_vlf', 'ECG_lf', 'ECG_lf_norm', 'ECG_hf', 'ECG_hf_norm', 'ECG_lf_hf_ratio']
    td_feat = pd.DataFrame(td_feature)
    fd_feat = pd.DataFrame(fd_feature)
    df_ecg = pd.concat([td_feat,fd_feat],axis = 1)
    df_ecg.columns = col
    df1 = pd.DataFrame([[np.nan] * len(df_ecg.columns)], columns=df_ecg.columns)
    df_ecg = df_ecg.append(df1, ignore_index=True)
    features.reset_index(inplace=True)
    temp = pd.concat([features, df_ecg], axis=1)
    temp.set_index(['subj','epoch'])
    
    
    df.append(temp)

df = pd.concat(df).set_index(['subj','epoch'])

  0%|          | 0/2 [00:00<?, ?it/s]

Extracting EDF parameters from /Users/yilanguo/Downloads/DSC180B/DSC180_sleep_apnea/data/edfs/shhs2/shhs2-200077.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/yilanguo/Downloads/DSC180B/DSC180_sleep_apnea/data/edfs/shhs2/shhs2-200078.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
skip due to different number of epoch


In [6]:
# Convert to category
df['stage'] = df['stage'].astype('category')

In [7]:
# Export
df.to_parquet(parent_dir+out_dir + "features_nsrr_shhs2.parquet")

In [8]:
df

age    eeg_abspow  eeg_abspow_c7min_norm  eeg_abspow_p2min_norm  \
subj   epoch                                                                    
200077 0       41    391.284180               0.013191               0.014578   
       1       41    262.349915               0.013696               0.010972   
       2       41    226.380569               0.014799               0.009100   
       3       41    282.347229               0.016450               0.008946   
       4       41    421.256561               0.018567               0.009365   
...           ...           ...                    ...                    ...   
       1538    41  17942.041016               0.984648               0.998910   
       1539    41  16969.201172               0.981720               0.985357   
       1540    41  16731.710938               0.978367               0.967267   
       1541    41  16767.021484               0.974755               0.949160   
       1542    41  16733.992188               0.970395               0.932270   

              eeg_alpha  eeg_alpha_c7min_norm  eeg_alpha_p2min_norm    eeg_at  \
subj   epoch                                                                    
200077 0       0.039596             -0.009696             -0.155950  0.723489   
       1       0.045965              0.010121             -0.134498  0.838666   
       2       0.074316              0.030474             -0.063685  0.994651   
       3       0.085423              0.045742             -0.009572  1.436497   
       4       0.087825              0.054556              0.071653  1.587354   
...                 ...                   ...                   ...       ...   
       1538    0.008848             -0.400324             -0.365248  0.260353   
       1539    0.009405             -0.399815             -0.363584  0.278189   
       1540    0.010130             -0.399147             -0.360736  0.274909   
       1541    0.009768             -0.398526             -0.358437  0.265126   
       1542    0.010177             -0.397757             -0.356199  0.259768   

              eeg_at_c7min_norm  eeg_at_p2min_norm  ...  ECG_SD1_SD2_ratio  \
subj   epoch                                        ...                      
200077 0               0.584591           0.242464  ...           0.526808   
       1               0.613832           0.294015  ...           0.480291   
       2               0.638720           0.357742  ...           0.414770   
       3               0.661299           0.488486  ...           0.407964   
       4               0.669169           0.681809  ...           0.375801   
...                         ...                ...  ...                ...   
       1538           -0.218489          -0.187056  ...           4.331897   
       1539           -0.216846          -0.178358  ...           4.335671   
       1540           -0.215230          -0.168793  ...           4.373404   
       1541           -0.213664          -0.163798  ...           4.282547   
       1542           -0.212001          -0.163929  ...                NaN   

               ECG_CSI   ECG_CVI  ECG_total_power       ECG_vlf    ECG_lf  \
subj   epoch                                                                
200077 0      1.898226 -1.220631              NaN           NaN       NaN   
       1      2.082070 -1.347927              NaN           NaN       NaN   
       2      2.410974 -1.358181              NaN           NaN       NaN   
       3      2.451200 -1.378949              NaN           NaN       NaN   
       4      2.660985 -1.395029         0.002282  3.456604e-04  0.001574   
...                ...       ...              ...           ...       ...   
       1538   0.230846  0.471608         0.043983  2.527806e-05  0.000551   
       1539   0.230645  0.470533         0.044394  1.263945e-05  0.000550   
       1540   0.228655  0.467386         0.043674  1.449359e-06  0.000539   
       1541   0.233506  0.476471         0.0

## Stats

In [9]:
# different because we skip one with different epoch #
df['stage'].value_counts(normalize=True, sort=True)

W     0.503564
N2    0.332469
R     0.112767
N1    0.028516
N3    0.022683
Name: stage, dtype: float64

In [10]:
df.groupby('stage')['eeg_iqr'].median()

stage
N1     11.556300
N2     13.835910
N3     20.791943
R       9.564116
W     226.539337
Name: eeg_iqr, dtype: float32

In [11]:
df.groupby('stage')['ECG_meanNN'].median()

stage
N1    0.903860
N2    0.949839
N3    0.952877
R     0.926422
W     0.935724
Name: ECG_meanNN, dtype: float64

In [12]:
df.groupby('stage')['ECG_CSI'].median()

stage
N1    4.509207
N2    3.784974
N3    2.790015
R     6.690143
W     0.319159
Name: ECG_CSI, dtype: float64